In [1]:
import pandas as pd
from surprise import Reader
from surprise.dataset import DatasetAutoFolds

from recommender.testing.custom_metric_utils import create_recommendations_profiles_embeddings, calculate_metric_scores
from recommender.testing.custom_metric_utils import create_users_profiles_embeddings
from recommender.tools.surprise_utils import train_model, SurpriseTests

In [2]:
DATA_ROOT = '../../data'

## Loading data

In [3]:
columns = ['bgg_user_name', 'bgg_id', 'bgg_user_rating']
test_df_implicit = pd.read_csv(f'{DATA_ROOT}/ratings_test_implicit.csv.gz')
train_df_implicit = pd.read_csv(f'{DATA_ROOT}/ratings_train_implicit.csv.gz')
test_df_explicit = pd.read_csv(f'{DATA_ROOT}/ratings_test_explicit.csv.gz')[columns]
train_df_explicit = pd.read_csv(f'{DATA_ROOT}/ratings_train_explicit.csv.gz')[columns]
games_df = pd.read_json(f'{DATA_ROOT}/bgg_GameItem.jl', lines=True)[[
    'name', 'bgg_id', 'mechanic', 'category', 'complexity',
    'max_players_best', 'min_players_best', 'max_players_rec', 'min_players_rec'
]]

features_names = pd.read_csv(f'{DATA_ROOT}/game_features_names.csv.gz').values.flatten()
game_features = pd.read_csv(f'{DATA_ROOT}/game_features.csv.gz')

In [4]:
users_profiles = pd.read_pickle(f'{DATA_ROOT}/users_profiles.pkl')
games_profiles = pd.read_pickle(f'{DATA_ROOT}/games_profiles.pkl')

In [5]:
mechanics_names = features_names[:20]
categories_names = features_names[20:40]

## Adding new users

In [6]:
USERS_TO_ADD = {
    # Family: Catan, Monopoly, UNO, Ticket to Ride
    "USER_1": {"games_ids": [13, 1406, 2223, 9209],
               "ratings": 4*[10.0]},
    # Negative interactions: Game of Thrones, War of the Ring, Twilight Imperium (3rd edition),
    # Battlestar Galactica, Scythe
    "USER_2": {"games_ids":[103343, 2228, 12493, 37111, 169786],
               "ratings": 5*[10.0]},
    # Worker placement: Viticulture, Agricola, Feast for Odin, West Kingdom
    "USER_3": {"games_ids":[183394, 31260, 177736, 236457, 266810, 296151],
               "ratings": 6*[10.0]},
    # Our games: Spirit Island, Citadel, Splendor, Avalon, Codenames, Crew, Love Letter, 7 wonders, viticulture, Descent
    "USER_4": {"games_ids":[162886, 478, 148228, 1862, 178900, 324856, 129622, 68448, 183394, 150586],
               "ratings": 10*[10.0]},
    # Ameritrash: Runewars, Arkham Horror, Starcraft, Last Night on Earth, Talisman, Divine Right 
    "USER_5": {"games_ids":[59294, 15987, 22827, 29368, 27627, 23],
               "ratings": 6*[10.0]},
    # Eurogames: Catan, Puerto Rico, Carcassone, Ra, El Grande, Five Tribes
    "USER_6": {"games_ids":[13, 3076, 822, 12, 93, 157354],
               "ratings": 6*[10.0]},
}

In [7]:
def append_users_to_train_df_explicit(train_df, users):
    for user, data in users.items():
        train_df = train_df.append(pd.DataFrame(data={'bgg_user_name': [user]*len(data['games_ids']), 'bgg_id': data['games_ids'], 'bgg_user_rating': data['ratings']}))
    return train_df

def append_users_to_train_df_implicit(train_df, users):
    for user, data in users.items():
        games = data['games_ids']
        train_df = train_df.append(pd.DataFrame(data={'bgg_user_name': [user]*len(games), 'bgg_id': games, 'value':[1]*len(games)}))
    return train_df

In [8]:
train_df_explicit = append_users_to_train_df_explicit(train_df_explicit, USERS_TO_ADD)
train_df_implicit = append_users_to_train_df_implicit(train_df_implicit, USERS_TO_ADD)

In [9]:
profiles_to_add = create_users_profiles_embeddings(
    train_df_implicit, games_df, categories_names, mechanics_names, 10, show_progress=True, users_subset=[f'USER_{i}' for i in range(1,7)]
)
users_profiles = users_profiles.append(profiles_to_add)

100%|██████████| 6/6 [00:00<00:00, 218.48it/s]


In [10]:
users_profiles.values[:, :40] *= 3

## Model training

In [11]:
reader = Reader(rating_scale=(1, 10))
trainset = DatasetAutoFolds.load_from_df(train_df_explicit, reader).build_full_trainset()

In [12]:
ALGO = 'SVD'
PARAMS = {
    'n_epochs': 16,
    'n_factors': 70,
}
model = train_model(ALGO, PARAMS, trainset, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15


## Inspecting predictions

In [13]:
tests = SurpriseTests(model, test_df_explicit, trainset)

In [14]:
def analyze(user_name):
    def game_id_to_name(game_id, games_df):
        game_info = games_df.loc[games_df['bgg_id'] == game_id]
        return game_info
    top_5 = tests.get_top_n_for_user_by_name(user_name=user_name, n=5)
    top_20 = tests.get_top_n_for_user_by_name(user_name=user_name, n=20)
    top_5_df = pd.DataFrame(top_5, columns = ['bgg_user_name', 'bgg_id', 'rating'])
    recommendations_profile = create_recommendations_profiles_embeddings(top_5_df, 5, games_df, categories_names, mechanics_names, 10)
    print("Metric value:", calculate_metric_scores(recommendations_profile, users_profiles.loc[user_name]))
    top_20_names = [game_id_to_name(prediction[1], games_df) for prediction in top_20]
    return pd.concat(top_20_names)

In [15]:
# Family: Catan, Monopoly, UNO, Ticket to Ride
analyze("USER_1")

Metric value: [0.28333333 1.02372881 0.9        0.         2.20706215]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
70395,Too Many Bones,192135,"[Cooperative Game:2023, Deck, Bag, and Pool Bu...","[Adventure:1022, Dice:1017, Fantasy:1010]",3.8586,3.0,3.0,4.0,1.0
15256,CATAN 3D Collector's Edition,17419,"[Dice Rolling:2072, Hand Management:2040, Memo...",[Negotiation:1026],2.6583,4.0,3.0,4.0,3.0
62819,Pandemic Legacy: Season 1,161936,"[Action Points:2001, Cooperative Game:2023, Ha...","[Environmental:1084, Medical:2145]",2.8336,4.0,4.0,4.0,2.0
37730,Cards Against Humanity,50381,"[Hand Management:2040, Player Judge:2865, Simu...","[Card Game:1002, Humor:1079, Mature / Adult:11...",1.1945,8.0,6.0,10.0,4.0
1864,Bridge,2181,"[Auction/Bidding:2012, Hand Management:2040, P...",[Card Game:1002],3.8562,4.0,4.0,4.0,4.0
64102,Nemesis,167355,"[Campaign / Battle Card Driven:2018, Cooperati...","[Adventure:1022, Horror:1024, Miniatures:1047,...",3.3735,4.0,4.0,5.0,1.0
3974,Pinochle,4583,"[Auction/Bidding:2012, Melding and Splaying:29...",[Card Game:1002],2.3023,4.0,4.0,4.0,3.0
57206,Carcassonne Big Box 3,141007,"[Area Majority / Influence:2080, Tile Placemen...","[City Building:1029, Medieval:1035]",2.0909,6.0,2.0,6.0,2.0
62356,Ticket to Ride: 10th Anniversary,160069,"[Card Drafting:2041, Connections:2883, End Gam...","[Trains:1034, Travel:1097]",1.9028,5.0,4.0,5.0,2.0
65360,Exploding Kittens: NSFW Deck,172242,"[Hand Management:2040, Player Elimination:2685...","[Animals:1089, Card Game:1002, Comic Book / St...",1.0798,5.0,4.0,5.0,3.0


In [16]:
# Negative interactions: Game of Thrones, War of the Ring, Twilight Imperium (3rd edition),
# Battlestar Galactica, Scythe
analyze("USER_2")

Metric value: [0.26714286 1.67920094 0.34285714 0.21428571 2.50348665]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
62819,Pandemic Legacy: Season 1,161936,"[Action Points:2001, Cooperative Game:2023, Ha...","[Environmental:1084, Medical:2145]",2.8336,4.0,4.0,4.0,2.0
78125,Twilight Imperium: Fourth Edition,233078,"[Action Drafting:2838, Area Majority / Influen...","[Civilization:1015, Economic:1021, Exploration...",4.2416,6.0,6.0,6.0,3.0
39321,War of the Ring Collector's Edition,60153,"[Area Majority / Influence:2080, Area Movement...","[Adventure:1022, Dice:1017, Fantasy:1010, Mini...",3.9915,2.0,2.0,4.0,2.0
10825,Twilight Struggle,12333,"[Action/Event:2840, Advantage Token:2847, Area...","[Modern Warfare:1069, Political:1001, Wargame:...",3.5890,2.0,2.0,2.0,2.0
67696,The 7th Continent,180263,"[Cooperative Game:2023, Grid Movement:2676, Ha...","[Adventure:1022, Card Game:1002, Exploration:1...",2.8821,2.0,1.0,3.0,1.0
69475,Star Wars: Rebellion,187645,"[Area Majority / Influence:2080, Area Movement...","[Civil War:1102, Fighting:1046, Miniatures:104...",3.7252,2.0,2.0,2.0,2.0
95463,Pandemic Legacy: Season 0,314040,"[Action Points:2001, Cooperative Game:2023, Ha...",[Spies/Secret Agents:1081],3.0147,4.0,2.0,4.0,2.0
84570,Tainted Grail: The Fall of Avalon,264220,"[Area Movement:2046, Campaign / Battle Card Dr...","[Adventure:1022, Exploration:1020, Fantasy:101...",3.2897,2.0,1.0,3.0,1.0
81648,Barrage,251247,"[Action Drafting:2838, Contracts:2912, End Gam...","[Economic:1021, Environmental:1084, Industry /...",4.0422,4.0,4.0,4.0,2.0
66024,Gloomhaven,174430,"[Action Queue:2689, Action Retrieval:2839, Cam...","[Adventure:1022, Exploration:1020, Fantasy:101...",3.8695,3.0,3.0,4.0,1.0


In [17]:
# Worker placement: Viticulture, Agricola, Feast for Odin, West Kingdom
analyze("USER_3")

Metric value: [0.025      0.98272182 1.05       0.1        2.15772182]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
80878,Eclipse: Second Dawn for the Galaxy,246900,"[Alliances:2916, Area Majority / Influence:208...","[Civilization:1015, Science Fiction:1016, Spac...",3.5341,6.0,4.0,6.0,2.0
87376,Anachrony: Infinity Box,278292,"[Action Points:2001, Card Drafting:2041, Dice ...","[Economic:1021, Miniatures:1047, Science Ficti...",4.0000,4.0,1.0,4.0,1.0
88623,Kanban EV,284378,"[Action Points:2001, Hand Management:2040, Var...","[Economic:1021, Industry / Manufacturing:1088]",4.3391,3.0,3.0,4.0,1.0
82728,Sleeping Gods,255984,"[Cooperative Game:2023, Events:2850, Hand Mana...","[Adventure:1022, Exploration:1020, Nautical:1008]",3.1102,2.0,2.0,3.0,1.0
84570,Tainted Grail: The Fall of Avalon,264220,"[Area Movement:2046, Campaign / Battle Card Dr...","[Adventure:1022, Exploration:1020, Fantasy:101...",3.2897,2.0,1.0,3.0,1.0
64700,Scythe,169786,"[Action Drafting:2838, Card Play Conflict Reso...","[Economic:1021, Fighting:1046, Miniatures:1047...",3.4225,4.0,4.0,5.0,1.0
82638,Trickerion: Collector's Edition,255668,"[Action Points:2001, Simultaneous Action Selec...",NaN,4.5513,4.0,3.0,4.0,1.0
90208,Gloomhaven: Jaws of the Lion,291457,"[Action Queue:2689, Action Retrieval:2839, Cam...","[Adventure:1022, Exploration:1020, Fantasy:101...",3.5705,4.0,1.0,4.0,1.0
65258,Orléans: Deluxe Edition,171905,"[Card Drafting:2041, Deck, Bag, and Pool Build...","[Medieval:1035, Religious:1115, Travel:1097]",3.0588,5.0,2.0,5.0,2.0
64579,Middara: Unintentional Malum – Act 1,169427,"[Action Points:2001, Cooperative Game:2023, Di...","[Adventure:1022, Exploration:1020, Fantasy:101...",3.9505,4.0,4.0,4.0,1.0


In [18]:
# Our games: Spirit Island, Citadel, Splendor, Avalon, Codenames, Crew, Love Letter, 7 wonders, viticulture, Descent
analyze("USER_4")

Metric value: [0.16670976 1.834914   0.64285714 0.18461538 2.82909628]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
53662,Viticulture,128621,"[Hand Management:2040, Turn Order: Progressive...","[Economic:1021, Farming:1013]",2.9321,4.0,3.0,5.0,2.0
64700,Scythe,169786,"[Action Drafting:2838, Card Play Conflict Reso...","[Economic:1021, Fighting:1046, Miniatures:1047...",3.4225,4.0,4.0,5.0,1.0
38362,Kingdom Death: Monster,55690,"[Cooperative Game:2023, Dice Rolling:2072, Gri...","[Adventure:1022, Fantasy:1010, Horror:1024, Ma...",4.2590,4.0,4.0,4.0,1.0
62819,Pandemic Legacy: Season 1,161936,"[Action Points:2001, Cooperative Game:2023, Ha...","[Environmental:1084, Medical:2145]",2.8336,4.0,4.0,4.0,2.0
75726,Pandemic Legacy: Season 2,221107,"[Cooperative Game:2023, Hand Management:2040, ...","[Environmental:1084, Exploration:1020, Medical...",3.2292,4.0,4.0,4.0,2.0
90208,Gloomhaven: Jaws of the Lion,291457,"[Action Queue:2689, Action Retrieval:2839, Cam...","[Adventure:1022, Exploration:1020, Fantasy:101...",3.5705,4.0,1.0,4.0,1.0
78125,Twilight Imperium: Fourth Edition,233078,"[Action Drafting:2838, Area Majority / Influen...","[Civilization:1015, Economic:1021, Exploration...",4.2416,6.0,6.0,6.0,3.0
59814,Shadows of Brimstone: Swamps of Death,150997,"[Cooperative Game:2023, Dice Rolling:2072, Gri...","[Adventure:1022, American West:1055, Explorati...",3.3929,4.0,1.0,4.0,1.0
82062,Cthulhu: Death May Die,253344,"[Action Points:2001, Cooperative Game:2023, Di...","[Fantasy:1010, Fighting:1046, Horror:1024, Min...",2.3784,3.0,3.0,4.0,1.0
96630,Everdell: Collector's Edition,319422,"[Card Drafting:2041, End Game Bonuses:2875, Ha...","[Animals:1089, Card Game:1002, City Building:1...",2.8571,4.0,1.0,4.0,1.0


In [19]:
# Ameritrash: Runewars, Arkham Horror, Starcraft, Last Night on Earth, Talisman, Divine Right
analyze("USER_5")

Metric value: [0.16       2.43789136 0.26       0.02       2.87789136]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
10959,Twilight Imperium: Third Edition,12493,"[Action Points:2001, Area-Impulse:2021, Dice R...","[Civilization:1015, Negotiation:1026, Politica...",4.2535,6.0,6.0,6.0,4.0
64700,Scythe,169786,"[Action Drafting:2838, Card Play Conflict Reso...","[Economic:1021, Fighting:1046, Miniatures:1047...",3.4225,4.0,4.0,5.0,1.0
66024,Gloomhaven,174430,"[Action Queue:2689, Action Retrieval:2839, Cam...","[Adventure:1022, Exploration:1020, Fantasy:101...",3.8695,3.0,3.0,4.0,1.0
67696,The 7th Continent,180263,"[Cooperative Game:2023, Grid Movement:2676, Ha...","[Adventure:1022, Card Game:1002, Exploration:1...",2.8821,2.0,1.0,3.0,1.0
38362,Kingdom Death: Monster,55690,"[Cooperative Game:2023, Dice Rolling:2072, Gri...","[Adventure:1022, Fantasy:1010, Horror:1024, Ma...",4.2590,4.0,4.0,4.0,1.0
90208,Gloomhaven: Jaws of the Lion,291457,"[Action Queue:2689, Action Retrieval:2839, Cam...","[Adventure:1022, Exploration:1020, Fantasy:101...",3.5705,4.0,1.0,4.0,1.0
78125,Twilight Imperium: Fourth Edition,233078,"[Action Drafting:2838, Area Majority / Influen...","[Civilization:1015, Economic:1021, Exploration...",4.2416,6.0,6.0,6.0,3.0
73141,Rising Sun,205896,"[Action Drafting:2838, Alliances:2916, Area Ma...","[Bluffing:1023, Fantasy:1010, Miniatures:1047,...",3.2832,5.0,4.0,6.0,3.0
75384,UBOOT: The Board Game,219100,"[Action Points:2001, Cooperative Game:2023, Si...","[Miniatures:1047, Nautical:1008, Real-time:103...",4.0873,4.0,4.0,4.0,1.0
43950,Xia: Legends of a Drift System,82222,"[Action Points:2001, Dice Rolling:2072, Grid M...","[Adventure:1022, Exploration:1020, Fighting:10...",3.1738,3.0,3.0,4.0,3.0


In [20]:
# Eurogames: Catan, Puerto Rico, Carcassone, Ra, El Grande, Five Tribes
analyze("USER_6")

Metric value: [0.12631579 1.28805618 1.23157895 0.09473684 2.74068776]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
48638,Puerto Rico (with two expansions),108687,"[Action Drafting:2838, End Game Bonuses:2875, ...","[City Building:1029, Economic:1021, Farming:1013]",3.2685,4.0,4.0,5.0,2.0
65260,El Grande Big Box,171908,"[Area Majority / Influence:2080, Area Movement...",[Renaissance:1070],2.7742,5.0,5.0,5.0,3.0
72165,Agricola (Revised Edition),200680,"[Enclosure:2043, Hand Management:2040, Worker ...","[Animals:1089, Economic:1021, Farming:1013]",3.5147,4.0,4.0,4.0,1.0
87376,Anachrony: Infinity Box,278292,"[Action Points:2001, Card Drafting:2041, Dice ...","[Economic:1021, Miniatures:1047, Science Ficti...",4.0000,4.0,1.0,4.0,1.0
39730,Dominant Species,62219,"[Action Queue:2689, Area Majority / Influence:...","[Animals:1089, Environmental:1084, Prehistoric...",4.0416,4.0,4.0,6.0,3.0
44240,The Castles of Burgundy,84876,"[Dice Rolling:2072, Grid Coverage:2978, Hexago...","[Dice:1017, Medieval:1035, Territory Building:...",2.9992,2.0,2.0,4.0,2.0
58867,El Grande Decennial Edition,147170,"[Area Majority / Influence:2080, Auction/Biddi...","[Medieval:1035, Political:1001]",2.9688,5.0,2.0,5.0,2.0
51423,Terra Mystica,120677,"[End Game Bonuses:2875, Hexagon Grid:2026, Inc...","[Civilization:1015, Economic:1021, Fantasy:101...",3.9633,4.0,4.0,5.0,2.0
2279,Power Grid,2651,"[Auction/Bidding:2012, Auction: Turn Order Unt...","[Economic:1021, Industry / Manufacturing:1088]",3.2682,5.0,4.0,6.0,3.0
73790,Dominion (Second Edition),209418,"[Card Drafting:2041, Deck, Bag, and Pool Build...","[Card Game:1002, Medieval:1035]",2.1712,3.0,2.0,4.0,2.0
